In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
os.chdir("../..")


In [7]:
import numpy as np

from enum import Enum

from typing import Callable

from qiskit import QuantumCircuit
from qiskit.circuit.gate import Gate as QkGate
from qiskit.circuit.library import RXGate, RYGate, RZGate, CZGate
from qiskit.circuit import Parameter, Instruction


In [4]:
from qml.tools.random import XRandomGenerator


In [5]:
def single_qubit_qargs(qc: QuantumCircuit, qubit_idx: int) -> list:
    return [qc.qubits[qubit_idx]]


def generate_double_qubit_args(spread: int = 1) -> Callable:
    def double_qubit_args(qc: QuantumCircuit, qubit_idx: int) -> list:
        return [
            qc.qubits[qubit_idx],
            qc.qubits[(qubit_idx+spread) % qc.num_qubits],
        ]
    return double_qubit_args


In [30]:
class GateInfo(Enum):

    RX = (RXGate, True, False, single_qubit_qargs)
    RY = (RYGate, True, False, single_qubit_qargs)
    RZ = (RZGate, True, False, single_qubit_qargs)
    CZ1 = (CZGate, False, True, generate_double_qubit_args(1))

    def __init__(
            self,
            gate_class: QkGate,
            trainable: bool,
            multibit: bool,
            qargs: Callable,
    ):
        self.gate_class = gate_class
        self.trainable = trainable
        self.multibit = multibit
        self.qargs = qargs
    
    @classmethod
    def get(cls, obj):
        if isinstance(obj, Gateset):
            return obj
        if not isinstance(obj, str):
            raise ValueError(f"Gateset.get() requires Gateset or string, but {type(obj)} is given.")
        name = obj.upper()
        for g in cls:
            if name == g.name:
                return g
        raise ValueError(f"Incorrect obj is given; {obj}")
    

class Gateset:

    GATE_LIST = dict()

    def __init__(self, num_qubits: int, seed: int = None):
        self._rng = XRandomGenerator(seed)
        self._nq = num_qubits

        for key, gate in self.GATE_LIST.items():
            self.__setattr__(key, gate)

        # accessors
            self.keys = self.GATE_LIST.keys
            self.items = self.GATE_LIST.items
            self.values = self.GATE_LIST.values
    
    @staticmethod
    def set_num_qubit(num_qubit: int, seed: int = None) -> "Gateset":
        if num_qubit == 2:
            return Gateset2Qubit(seed=seed)
        if num_qubit == 3:
            return Gateset3Qubit
        raise ValueError(f'num_qubit {num_qubit} is not supported.')
    
    def __iter__(self):
        return iter(self.GATE_LIST.values())
    
    def get(self, obj):
        if isinstance(obj, GateInfo):
            return obj
        obj = obj.upper()
        if obj not in self.GATE_LIST:
            raise ValueError("Undefined gate is attempted to retrieve.")
        return self.GATE_LIST[obj]
    
    def get_at_random(self):
        return self._rng.choice([g for g in self])


class Gateset2Qubit(Gateset):

    NUM_QUBITS = 2

    GATE_LIST = dict(
        RX=GateInfo.RX,
        RY=GateInfo.RY,
        RZ=GateInfo.RZ,
        CZ=GateInfo.CZ1,
    )

    def __init__(self, seed: int = None):
        super().__init__(self.NUM_QUBITS, seed=seed)


class Gateset3Qubit(Gateset):

    NUM_QUBITS = 3

    GATE_LIST = dict(
        RX=GateInfo.RX,
        RY=GateInfo.RY,
        RZ=GateInfo.RZ,
        CZ=GateInfo.CZ1,
    )

    def __init__(self, seed: int = None):
        super().__init__(self.NUM_QUBITS, seed=seed)


In [31]:
gset = Gateset.set_num_qubit(2)
gset.get_at_random()


<GateInfo.CZ1: (<class 'qiskit.circuit.library.standard_gates.z.CZGate'>, False, True, <function generate_double_qubit_args.<locals>.double_qubit_args at 0x10fdeb600>)>

In [10]:
class Gate:

    def __init__(
            self,
            gate: Instruction,
            trainable: bool,
            multibit: bool,
            qubit: int,
            qargs_func: Callable,
    ):
        self._gate = gate
        self._trainable = trainable
        self._multibit = multibit
        self._qubit = qubit
        self._qargs_func = qargs_func

    def apply_to_qc(self, qc: QuantumCircuit):
        qargs = self._qargs_func(qc, self._qubit)
        qc.append(self._gate, qargs)

    @staticmethod
    def new_with_info(info: GateInfo, qubit: int, parameter: Parameter = None):
        gate = info.gate_class() if parameter is None else info.gate_class(parameter)
        return Gate(
            gate, info.trainable, info.multibit, qubit, info.qargs
        )

    @property
    def gate(self):
        return self._gate

    @property
    def trainable(self):
        return self._trainable

    @property
    def multi_qubit(self):
        return self._multi_qubit

    @property
    def qubit(self):
        return self._qubit
